## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-25-18-55-03 +0000,bbc,220 MPs call for Starmer to recognise Palestin...,https://www.bbc.com/news/articles/cx202zvygmlo
1,2025-07-25-18-53-08 +0000,bbc,Almost a third of people in Gaza not eating fo...,https://www.bbc.com/news/articles/ckgjg81qqwvo
2,2025-07-25-18-51-49 +0000,nypost,‘Doomsday mom’ Lori Vallow Daybell sentenced t...,https://nypost.com/2025/07/25/us-news/doomsday...
3,2025-07-25-18-50-06 +0000,startribune,Education Department says it will release bill...,https://www.startribune.com/education-departme...
4,2025-07-25-18-45-51 +0000,nyt,How Will David Ellison Tackle the Big Problems...,https://www.nytimes.com/2025/07/25/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
57,trump,62
6,gaza,19
211,new,16
97,israel,14
65,talks,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
229,2025-07-25-01-00-00 +0000,wsj,Streetwise: The Trump administration has rampe...,https://www.wsj.com/economy/central-banking/tr...,150
84,2025-07-25-15-18-56 +0000,bbc,Trump says Hamas 'didn't want deal' as US and ...,https://www.bbc.com/news/articles/crrqrz857wgo,136
230,2025-07-25-01-00-00 +0000,wsj,Buoyed by voters’ improving views of the econo...,https://www.wsj.com/politics/policy/trump-appr...,123
219,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...,122
241,2025-07-24-23-54-32 +0000,nyt,Powell Tells Trump He Was Wrong About Renovati...,https://www.nytimes.com/2025/07/24/us/politics...,119


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
229,150,2025-07-25-01-00-00 +0000,wsj,Streetwise: The Trump administration has rampe...,https://www.wsj.com/economy/central-banking/tr...
312,103,2025-07-24-19-57-36 +0000,cbc,"Israel, U.S. pull out of Gaza ceasefire talks ...",https://www.cbc.ca/news/world/france-macron-pa...
230,61,2025-07-25-01-00-00 +0000,wsj,Buoyed by voters’ improving views of the econo...,https://www.wsj.com/politics/policy/trump-appr...
249,51,2025-07-24-23-15-00 +0000,wsj,Top Justice Department Official Visits Former ...,https://www.wsj.com/us-news/law/top-justice-de...
59,45,2025-07-25-16-32-14 +0000,nyt,Thailand Warns of War With Cambodia as Deadly ...,https://www.nytimes.com/2025/07/25/world/asia/...
265,37,2025-07-24-22-24-35 +0000,nypost,FCC greenlights $8.4B sale of CBS parent Param...,https://nypost.com/2025/07/24/media/fcc-greenl...
219,37,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...
57,35,2025-07-25-16-36-09 +0000,nypost,Trump forecasts ‘50-50 chance’ for EU trade de...,https://nypost.com/2025/07/25/us-news/trump-fo...
21,34,2025-07-25-18-11-18 +0000,nypost,Kaylee Goncalves’ sister recounts chilling mom...,https://nypost.com/2025/07/25/us-news/kaylee-g...
63,34,2025-07-25-16-25-00 +0000,wsj,President Trump has long wanted a U.S. soverei...,https://www.wsj.com/politics/policy/sovereign-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
